# Методы для работы с текстом

Для стандартных строк (strings) Python доступно много методов для обработки текста:

In [2]:
mystring = 'hello'

In [3]:
mystring.capitalize()

'Hello'

In [4]:
mystring.isdigit()

False

In [5]:
help(str)

Help on class str in module builtins:

class str(object)
 |  str(object='') -> str
 |  str(bytes_or_buffer[, encoding[, errors]]) -> str
 |  
 |  Create a new string object from the given object. If encoding or
 |  errors is specified, then the object must expose a data buffer
 |  that will be decoded using the given encoding and error handler.
 |  Otherwise, returns the result of object.__str__() (if defined)
 |  or repr(object).
 |  encoding defaults to sys.getdefaultencoding().
 |  errors defaults to 'strict'.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __format__(self, format_spec, /)
 |      Return a formatted version of the string as described by format_spec.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  

# Pandas и текст

Pandas может сделать с текстом намного больше. Полная документация по таким продвинутым методам, как индексация строк и регулярные выражения в pandas, доступна по следующей ссылке: https://pandas.pydata.org/docs/user_guide/text.html

## Методы Pandas для работы с текстом в строковой (текстовой) колонке

In [6]:
import pandas as pd

In [7]:
names = pd.Series(['andrew','bobo','claire','david','4'])

In [8]:
names

0    andrew
1      bobo
2    claire
3     david
4         4
dtype: object

In [9]:
names.str.capitalize()

0    Andrew
1      Bobo
2    Claire
3     David
4         4
dtype: object

In [10]:
names.str.isdigit()

0    False
1    False
2    False
3    False
4     True
dtype: bool

## Разбиение строки на части, извлечение отдельных частей из строки (string)

In [14]:
tech_finance = ['GOOG,APPL,AMZN','JPM,BAC,GS']

In [15]:
len(tech_finance)

2

In [16]:
tickers = pd.Series(tech_finance)

In [17]:
tickers

0    GOOG,APPL,AMZN
1        JPM,BAC,GS
dtype: object

In [18]:
tickers.str.split(',')

0    [GOOG, APPL, AMZN]
1        [JPM, BAC, GS]
dtype: object

In [19]:
tickers.str.split(',').str[0]

0    GOOG
1     JPM
dtype: object

In [21]:
tickers.str.split(',',expand=True)

0     1     2
0  GOOG  APPL  AMZN
1   JPM   BAC    GS

## Очистка и редактирование строк

In [22]:
messy_names = pd.Series(["andrew  ","bo;bo","  claire  "])

In [27]:
# Обратите внимание на "сдвиги строк" из-за дополнителных пробелов 
# в значениях "andrew  " и "  claire  "
messy_names

0      andrew  
1         bo;bo
2      claire  
dtype: object

In [28]:
messy_names.str.replace(";","")

0      andrew  
1          bobo
2      claire  
dtype: object

In [29]:
messy_names.str.strip()

0    andrew
1     bo;bo
2    claire
dtype: object

In [31]:
messy_names.str.replace(";","").str.strip()

0    andrew
1      bobo
2    claire
dtype: object

In [32]:
messy_names.str.replace(";","").str.strip().str.capitalize()

0    Andrew
1      Bobo
2    Claire
dtype: object

## Альтернатива - использование apply() со своей собственной функцией

In [33]:
def cleanup(name):
    name = name.replace(";","")
    name = name.strip()
    name = name.capitalize()
    return name

In [34]:
messy_names

0      andrew  
1         bo;bo
2      claire  
dtype: object

In [35]:
messy_names.apply(cleanup)

0    Andrew
1      Bobo
2    Claire
dtype: object

## Какой вариант более эффективен?

In [43]:
import timeit 
  
# Этот фрагмент кода запускается только один раз
setup = '''
import pandas as pd
import numpy as np
messy_names = pd.Series(["andrew  ","bo;bo","  claire  "])
def cleanup(name):
    name = name.replace(";","")
    name = name.strip()
    name = name.capitalize()
    return name
'''
  
# Это фрагменты кода, для которых мы будем измерять время выполнения
# (они запускаются много раз)
stmt_pandas_str = ''' 
messy_names.str.replace(";","").str.strip().str.capitalize()
'''

stmt_pandas_apply = '''
messy_names.apply(cleanup)
'''

stmt_pandas_vectorize='''
np.vectorize(cleanup)(messy_names)
'''

In [44]:
timeit.timeit(setup = setup, 
                    stmt = stmt_pandas_str, 
                    number = 10000) 

3.931618999999955

In [45]:
timeit.timeit(setup = setup, 
                    stmt = stmt_pandas_apply, 
                    number = 10000) 

1.2268500999999787

In [46]:
timeit.timeit(setup = setup, 
                    stmt = stmt_pandas_vectorize, 
                    number = 10000) 

0.28283379999993485

Вот это да! Хотя методы .str() могут быть очень удобными, когда дело доходит до повышения производительности, не забывайте про np.vectorize()! Вспомните лекцию "Полезные методы", в которой мы обсуждали np.vectorize()